# Project 1

## (Subject)

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import locale
import math
from countries import countries as c_list
import statsmodels.api as sm

## Import data

In [2]:
# Full-code example
# GDP_data = pd.read_csv("Data/SYB62_T13_201904_GDP_and_GDP_Per_Capita.csv", encoding="ISO-8859-1", skiprows=[0])

# Function to import
def myImport(target):
    out = pd.read_csv(target, skiprows = [0], encoding="ISO-8859-1")
    return out
    
LE_df = myImport("Data/SYB60_T03_Population Growth_Fertility_and_Mortality_Indicators.csv")
GDP_df = myImport("Data/SYB62_T13_201904_GDP_and_GDP_Per_Capita.csv")
POP_df = myImport("Data/SYB61_T02_Population_Surface_Area_and_Density.csv")
POPg_df = myImport("Data/SYB61_T03_Population_Growth_Rates_in_Urban_areas_and_Capital_cities.csv")
NRG_df = myImport( "Data/SYB62_T23_201904_Production_Trade_and_Supply_of_Energy.csv")

## Helper functions

In [7]:
# Original code:
#LE_filtered_df = LE_df.loc[(LE_df["Series"] == "Life expectancy at birth for both sexes (years)")]\
#    .drop(columns=["Footnotes", "Source", "Series"])\
#    .rename(columns={"Unnamed: 1": "Place",
#                     "Value": "Life Expectency"})

# Make it easier to filter data
def importDataFrame(df, series, name):
    global all_df
    
    # Build a drop list
    droplist = [var for var in df.columns if var not in ["Unnamed: 1", "Region/Country/Area", "Year", "Value"]]
    
    # Read in data, changing column names and keeping only desired data
    new = df.loc[df["Series"] == f"{series}"]\
        .drop(columns=droplist)\
        .rename(columns={"Unnamed: 1": "Place",
                         "Value": f"{name}"})\
        .set_index("Place", drop=True)\
        .loc[c_list]\
        .dropna()
    
    # Value read in is expected to be numeric.  Process as needed to return numeric
    new[f"{name}"] = new[f"{name}"]\
        .transform(lambda x: x.replace(',', ''))\
        .apply(float)    
    
    if "all_df" not in globals():
        all_df = new
    elif f"{name}" not in all_df.columns.to_list():
        left = all_df.reset_index()
        right = new.reset_index()
        
        all_df = pd.merge(left, right, on=["Place", "Region/Country/Area", "Year"], how="outer").set_index("Place")
    
    return new

# Get top and bottom in category
def getTopAndBottom(value, df=None, num=5, year=2015):
    if df == None and "all_df" in globals():
        df = all_df
    
    top = df.loc[df["Year"] == year]\
        .sort_values(f"{value}")\
        .tail(num)\
        .index\
        .to_list()

    bottom = df.loc[df["Year"] == year]\
        .sort_values(f"{value}")\
        .head(num)\
        .index\
        .to_list()
    
    return (top, bottom)

# Get bounds for plot.  Use off to apply an offset for graphing purposes
def getBounds(value, df=None, off=0):
    if df == None and "all_df" in globals():
        df = all_df
        
    minval = df.sort_values(f"{value}").iloc[0][f"{value}"]
    maxval = df.sort_values(f"{value}").iloc[-1][f"{value}"]
    
    off_minval = minval - (maxval - minval) * off
    off_maxval = maxval + (maxval - minval) * off
    
    return (off_minval, off_maxval)

# Can probably get rid of this
# Get bounds for year-dependent plot
def getYearPlotBounds(value, df=None, off_pct=0.05):
    if df == None and "all_df" in globals():
        df = all_df
    
    # Exact bounds
    min_x = df.sort_values("Year").iloc[0]["Year"]
    max_x = df.sort_values("Year").iloc[-1]["Year"]
    min_y = df.sort_values(f"{value}").iloc[0][f"{value}"]
    max_y = df.sort_values(f"{value}").iloc[-1][f"{value}"]
    
    # Offsets
    off_min_x = min_x - (max_x - min_x) * off_pct
    off_max_x = max_x + (max_x - min_x) * off_pct
    off_min_y = min_y - (max_y - min_y) * off_pct
    off_max_y = max_y + (max_y - min_y) * off_pct
    
    return ([min_x, max_x, min_y, max_y],
            [off_min_x, off_max_x, off_min_y, off_max_y])

# Make the plots

def myScatter(value1, label1, value2, label2, title, saveas, df1, df2, 
              off_pct1=0.05, off_pct2=0.05):

    # Parameters denoted with a 1 correspond to x-axis; 2 represents y-axis

    # Initialize graph
    fig, ax = plt.subplots()
    fig.set_figheight(10)
    fig.set_figwidth(20)
    ax.grid()
    
    # Get axis limits
    x_exact, x_off = getYearPlotBounds(value1, df=df1, off_pct=off_pct1)
    y_exact, y_off = getYearPlotBounds(value2, df=df2, off_pct=off_pct2)
    
    ax.set_xlim((x_off[2], x_off[3]))
    ax.set_ylim((y_off[2], y_off[3]))
    
    ax.set_xlabel(f"{label1}", fontsize=18)
    ax.set_ylabel(f"{label2}", fontsize=18)
    ax.set_title(f"{title}", fontsize=25)
    ax.tick_params(labelsize=14)

    # Plot one year
    ax.scatter(df1.loc[df1["Year"] == 2005][f"{value1}"],
               df2.loc[df2["Year"] == 2005][f"{value2}"],
               s = 75,
               c = "red",
               alpha = 0.5,
               marker = "o",
               label="2005")

    # Plot one year
    ax.scatter(df1.loc[df1["Year"] == 2010][f"{value1}"],
               df2.loc[df2["Year"] == 2010][f"{value2}"],
               s = 75,
               c = "red",
               alpha = 0.5,
               marker = "o",
               label="2010")

    # Plot one year
    ax.scatter(df1.loc[df1["Year"] == 2015][f"{value1}"],
               df2.loc[df2["Year"] == 2015][f"{value2}"],
               s = 75,
               c = "red",
               alpha = 0.5,
               marker = "o",
               label="2015")

    ax.legend(fontsize=12)

    plt.savefig(f"{saveas}.png")
    
    return fig, ax

## Pull in all data

In [8]:
# Build master df

LE_filtered_df = importDataFrame(LE_df, "Life expectancy at birth for both sexes (years)", "Life Expectancy")
GDP_filtered_df = importDataFrame(GDP_df, "GDP per capita (US dollars)", "GDP")
POP_filtered_df = importDataFrame(POP_df, "Population mid-year estimates (millions)", "Total Population")
POPg_filtered_df = importDataFrame(POPg_df, "Urban population (percent)", "Urban Population")
NRG_filtered_df = importDataFrame(NRG_df, "Primary energy production (petajoules)", "Energy Production")

all_df.head()

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,Region/Country/Area,Year,Life Expectancy,GDP,Total Population,Urban Population,Energy Production
Place,,,,,,,
Afghanistan,4.0,2005.0,56.9,264.0,25.07,22.7,23.0
Afghanistan,4.0,2010.0,60.0,558.0,28.80,23.7,41.0
Afghanistan,4.0,2015.0,62.3,611.0,33.74,24.8,60.0
Albania,8.0,2005.0,74.8,2615.0,3.08,46.7,48.0
Albania,8.0,2010.0,75.6,4056.0,2.94,52.2,69.0


## Other derived values of interest

In [11]:
all_df["Log GDP"] = all_df["GDP"].apply(math.log)

In [ ]:
# Test scatter